In [1]:
import os
import time
import pandas as pd
import numpy as np
from multiprocessing import Pool
import boto3
from s3fs.core import S3FileSystem
import matplotlib.pyplot as plt
from scipy import stats
import re




In [2]:
s3 = boto3.resource('s3')
client = boto3.client('s3')
bucket = s3.Bucket('usa-edges-for-speed-mapping')

In [16]:
wiki = pd.read_excel('speedLimits_wiki_02.xlsx', 
             header=1)
wiki = wiki.drop([1, 2, 12, 25, 38, 43, 49, 53])
wiki = wiki.reset_index()


regex = re.compile(".*?\[(.*?)\]")

for j in range(2, 8):
    for i in range(0, len(wiki)):
        loc = wiki.iloc[i, j].find('(')
        wiki.iloc[i, j] = wiki.iloc[i, j][:loc-5]
        if wiki.iloc[i, j].find('[') != -1:
            result = re.findall(regex, wiki.iloc[i, j])
            wiki.iloc[i, j] = wiki.iloc[i, j].replace(result[0], '')
            wiki.iloc[i, j] = wiki.iloc[i, j].replace('[', '')
            wiki.iloc[i, j] = wiki.iloc[i, j].replace(']', '')
        if i != 7:
            if len(wiki.iloc[i, j]) > 2:
                wiki.iloc[i, j] = wiki.iloc[i, j][-2:]
wiki


,index,State or territory,Freeway (rural),Freeway (trucks),Freeway (urban),Divided (rural),Undivided (rural),Residential
0,0,Alabama,70,70,70,65,65,25
1,3,Arizona,75,75,65,65,65,25
2,4,Arkansas,75,70,65,65,65,30
3,5,California,70,55,65,65,65,30
4,6,Colorado,75,75,75,65,65,35
5,7,Connecticut,65,65,55,55,50,40
6,8,Delaware,65,65,65,60,50,35
7,9,District of Columbia,no rural freeways i,no rural freeways i,55,no rural roads i,no rural roads i,25
8,10,Florida,70,70,70,65,60,50
9,11,Georgia,70,70,70,65,55,45


In [17]:
names = ['Alabama',
'Arizona',
'Arkansas',
'California',
'Colorado',
'Connecticut',
'Delaware',
'Distric of Columbia',
'Florida',
'Georgia',
'Hawaii',
'Idaho',
'Illinois',
'Indiana',
'Iowa',
'Kansas',
'Kentucky',
'Louisiana',
'Maine',
'Maryland',
'Massachusetts',
'Michigan',
'Minnesota',
'Mississippi',
'Missouri',
'Montana',
'Nebraska',
'Nevada',
'New Hampshire',
'New Jersey',
'New Mexico',
'New York',
'North Carolina',
'North Dakota',
'Ohio',
'Oklahoma',
'Oregon',
'Pennsylvania',
'Rhode Island',
'South Carolina',
'South Dakota',
'Tennessee',
'Texas',
'Utah',
'Vermont',
'Virginia',
'Washington',
'West Virginia',
'Wisconsin',
'Wyoming']


abbrv = ['AL',
'AZ',
'AR',
'CA',
'CO',
'CT',
'DE',
'DC',
'FL',
'GA',
'HI',
'ID',
'IL',
'IN',
'IA',
'KS',
'KY',
'LA',
'ME',
'MD',
'MA',
'MI',
'MN',
'MS',
'MO',
'MT',
'NE',
'NV',
'NH',
'NJ',
'NM',
'NY',
'NC',
'ND',
'OH',
'OK',
'OR',
'PA',
'RI',
'SC',
'SD',
'TN',
'TX',
'UT',
'VT',
'VA',
'WA',
'WV',
'WI',
'WY']

# Get Original Edge files from S3

In [3]:
edge_files = []
for key in bucket.objects.filter(Prefix='edges/'):
    edge_files.append(key.key)
#edges = edges[1:]
print('There are ', len(edge_files), 'edge files.')

edges_csv = []
for i in range(0, len(edge_files)):
    edges_csv.append(edge_files[i].replace('edges/',''))
edges_csv
n_csvs = len(edges_csv)

There are  3110 edge files.


# Enrich files by State

In [50]:
def enrichCSV(n):
    
    try:
        # Get Edges file from S3...
        s3fileSys = S3FileSystem(anon=False)
        key_sys = edge_files[n]
        bucket_sys = 'usa-edges-for-speed-mapping'
        edges = pd.read_csv(s3fileSys.open('{}/{}'.format(bucket_sys, key_sys),
                                mode='rb'), low_memory=False)

        edges['SpeedReplacedOSM'] = np.zeros(len(edges))
        edges['SpeedReplacedWiki'] = np.zeros(len(edges))
        edges['SpeedConfirmed'] = np.zeros(len(edges))
        
        # Check for missing fields, and discard unnecessary fields:
        edges_tmp = pd.concat([edges['highway'], edges['name']], axis = 1)
        try:
            edges_tmp = pd.concat([edges_tmp, edges['maxspeed']], axis = 1)
        except:
            edges_tmp['maxspeed'] = np.nan
        try:
            edges_tmp = pd.concat([edges_tmp, edges['lanes']], axis = 1)
        except:
            edges_tmp['lanes'] = np.nan
        try:
            edges_tmp = pd.concat([edges_tmp, edges['oneway']], axis = 1)
        except:
            edges_tmp['oneway'] = np.nan
        edges_tmp = pd.concat([edges_tmp, edges['SpeedReplacedOSM'], edges['SpeedReplacedWiki'], edges['SpeedConfirmed']], axis = 1)
        edges = edges_tmp
        del edges_tmp


        ihighway = edges.columns.get_loc('highway')
        ispeed = edges.columns.get_loc('maxspeed')
        ireplace = edges.columns.get_loc('SpeedReplacedOSM')

        highway_types = edges['highway'].values
        highway_types = np.unique(highway_types)

        # initialize for statistical mode
        speed_modes = []
        highway_types_speed_available = []
        
        # Need to find nested speed limit elements
        regex = re.compile(".*?\[(.*?)\]")

        # Get statistical modes per road type:
        for road in highway_types:
            try:
                speeds = edges[edges['highway']==road]['maxspeed'].dropna().reset_index()
                speed_dum = []
                for speed in speeds.maxspeed.values:
                    speed_dum.append(speed.replace(' mph', ''))
                for i in range(0, len(speed_dum)):
                    if speed_dum[i].find('[') != -1:
                        result = re.findall(regex, speed_dum[i])
                        speed_dum[i] = speed_dum[i].replace(result[0], result[0][-3:-1])
                        speed_dum[i] = speed_dum[i].replace('[','')
                        speed_dum[i] = speed_dum[i].replace(']','')
                farray = np.asfarray(speed_dum)
                mode = stats.mode(farray)
                speed_modes.append(mode[0][0])
                highway_types_speed_available.append(road)
            except:
                print('No Non-NaNs for '+road)

        # Convert speeds to floats, remove nested elements, and remove units. mph is implicit.
        indices = edges[edges['maxspeed'].notna()].index
        numNotNan = len(indices)

        for i in range(0, numNotNan):
            if edges.iloc[indices[i], ispeed] == str:
                edges.iloc[indices[i], ispeed] = edges.iloc[indices[i], ispeed].replace(' mph', '')
                if edges.iloc[indices[i], ispeed].find('[') != -1:
                    result = re.findall(regex, edges.iloc[indices[i], ispeed])
                    edges.iloc[indices[i], ispeed] = edges.iloc[indices[i], ispeed].replace(result[0], result[0][-3:-1])
                    edges.iloc[indices[i], ispeed] = edges.iloc[indices[i], ispeed].replace('[','')
                    edges.iloc[indices[i], ispeed] = edges.iloc[indices[i], ispeed].replace(']','')
                    edges.iloc[indices[i], ispeed] = edges.iloc[indices[i], ispeed][-2:]
                try:
                    edges.iloc[indices[i], ispeed] = float(edges.iloc[indices[i], ispeed]) 
                except:
                    edges.iloc[indices[i], ispeed] = np.nan

        indices = edges[edges['maxspeed'].isna()].index
        numNan = len(indices)

        # Replace speed limit nan's with given road type's mode value:
        for i in range(0, numNan):
            for j in range(0, len(highway_types_speed_available)):
                try:
                    if edges.iloc[indices[i], ihighway] == highway_types_speed_available[j]:
                        edges.iloc[indices[i], ispeed] = speed_modes[j]
                        edges.iloc[indices[i], ireplace] = 1
                except:
                    a=1


        
        # Find state matching file name:
        regex = "([A-Z]+[A-Z]+)" # matches CAPITALS_CAPITALS only
        result = re.findall(regex, edges_csv[n])

        for i in range(0, len(abbrv)):
            if result[0] == abbrv[i]:
                istate = i
                print('Using Wikipedia to map speeds for '+edges_csv[n]+' to state: '+names[i])

        # After OSM speedlimit interpolation, replace remaining nan's with wiki data.
        indices = edges[edges['maxspeed'].isna()].index
        #ihighway = edges.columns.get_loc('highway')
        ispeed = edges.columns.get_loc('maxspeed')
        ireplace = edges.columns.get_loc('SpeedReplacedWiki')

        ifwyRural = wiki.columns.get_loc('Freeway (rural)')
        idivRural = wiki.columns.get_loc('Divided (rural)')
        iundivRural = wiki.columns.get_loc('Undivided (rural)')
        ires = wiki.columns.get_loc('Residential')

        for i in range(0, len(indices)):

            road = edges.iloc[indices[i], ihighway]

            if road == 'motorway':
                edges.iloc[indices[i], ispeed] = float(wiki.iloc[istate, ifwyRural])
                edges.iloc[indices[i], ireplace] = 1
            elif road == 'trunk':
                edges.iloc[indices[i], ispeed] = float(wiki.iloc[istate, idivRural])
                edges.iloc[indices[i], ireplace] = 1
            elif road == 'primary':
                edges.iloc[indices[i], ispeed] = float(wiki.iloc[istate, iundivRural])
                edges.iloc[indices[i], ireplace] = 1
            elif road == 'secondary':
                edges.iloc[indices[i], ispeed] = 0.8*float(wiki.iloc[istate, iundivRural])
                edges.iloc[indices[i], ireplace] = 1
            elif road == 'tertiary':
                edges.iloc[indices[i], ispeed] = 0.8*float(wiki.iloc[istate, iundivRural])
                edges.iloc[indices[i], ireplace] = 1
            elif road == 'unclassified':
                edges.iloc[indices[i], ispeed] = 0.8*float(wiki.iloc[istate, iundivRural])
                edges.iloc[indices[i], ireplace] = 1
            elif road == 'residential':
                edges.iloc[indices[i], ispeed] = float(wiki.iloc[istate, ires])
                edges.iloc[indices[i], ireplace] = 1


        # Write file to S3...
        path = '/home/ec2-user/SageMaker/osm/edges_enriched/'+edges_csv[n].replace('.csv','_enriched.csv')
        edges.to_csv(path)
        del edges
        # write CSV to s3
        s3.meta.client.upload_file(path, 'usa-edges-for-speed-mapping', 'edges_enriched/{}'.format(edges_csv[n].replace('.csv','_enriched.csv')))
        print('Saved '+path+' to S3.')    
        s3dir = client.list_objects(Bucket='usa-edges-for-speed-mapping',
                    Prefix='edges_enriched/')
        # remove CSV from local EC2 instance.
        os.remove(path)
        files = []
        for key in bucket.objects.filter(Prefix='edges_enriched/'):
            files.append(key.key)
        print((len(files)-1)/len(edges_csv)*100, 'percent complete.')
        
    except:
        # track indices of errors...
        print('Error for n = ', n, edges_csv[n]+'.%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    

In [51]:
if __name__ == '__main__':
    with Pool(processes=6) as p:
        #p.map(enrichCSV, range(0,n_csvs))

# Rerun for missing n's due to crash:
        p.map(enrichCSV, isnotin)

No Non-NaNs for primary
No Non-NaNs for residential
No Non-NaNs for secondary
No Non-NaNs for ['unclassified', 'residential']
No Non-NaNs for tertiary
No Non-NaNs for primary
No Non-NaNs for trunk
No Non-NaNs for residential
No Non-NaNs for unclassified
No Non-NaNs for tertiary
Using Wikipedia to map speeds for NC_JonesCounty.csv to state: North Carolina
No Non-NaNs for unclassified
Using Wikipedia to map speeds for MT_WheatlandCounty.csv to state: Montana
No Non-NaNs for ['unclassified', 'residential', 'tertiary']
No Non-NaNs for ['unclassified', 'residential']
No Non-NaNs for ['unclassified', 'tertiary']
No Non-NaNs for primary
No Non-NaNs for primary_link
No Non-NaNs for residential
No Non-NaNs for secondary
No Non-NaNs for secondary_link
No Non-NaNs for tertiary
No Non-NaNs for unclassified
Using Wikipedia to map speeds for CA_KingsCounty.csv to state: California
No Non-NaNs for ['residential', 'unclassified']
No Non-NaNs for ['tertiary', 'secondary']
No Non-NaNs for primary
No Non

In [ ]:
enrichCSV(0)

## Check S3 for missing eniched files

In [5]:
files = []
for key in bucket.objects.filter(Prefix='edges_enriched/'):
    files.append(key.key)
files = files[1:]
len(files)

3104

In [6]:
files[0]

'edges_enriched/AR_ArkansasCounty_enriched.csv'

In [8]:
for i in range(0,len(files)):
    files[i] = files[i].replace('edges_enriched/','')
for i in range(0,len(files)):
    files[i] = files[i].replace('_enriched','')
files[0]

'AR_ArkansasCounty.csv'

In [20]:
isin = []
isnotin = []
for n in range(0, n_csvs):
    if edges_csv[n] in files:
        #print(n)
        isin.append(n)
    else:
        isnotin.append(n)
        print('No match for n = ', n)

No match for n =  112
No match for n =  994
No match for n =  1503
No match for n =  1597
No match for n =  1659
No match for n =  2162


In [22]:
for n in isnotin:
    print(edges_csv[n])
    #print(edge_files[n])

CA_KingsCounty.csv
KY_KnoxCounty.csv
MS_LefloreCounty.csv
MT_WheatlandCounty.csv
NC_JonesCounty.csv
OR_BakerCounty.csv
